In [ ]:
import os
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from alpaca.trading.stream import TradingStream
from alpaca.trading.stream import TradeUpdate
import requests
from dotenv import load_dotenv
from zoneinfo import ZoneInfo
import json
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.corporate_actions import CorporateActionsClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.stock import StockDataStream
import mongo
import nest_asyncio
import asyncio

from alpaca.data.requests import (
    CorporateActionsRequest,
    StockBarsRequest,
    StockQuotesRequest,
    StockTradesRequest,
)
from alpaca.trading.requests import (
    ClosePositionRequest,
    GetAssetsRequest,
    GetOrdersRequest,
    LimitOrderRequest,
    MarketOrderRequest,
    StopLimitOrderRequest,
    StopLossRequest,
    StopOrderRequest,
    TakeProfitRequest,
    TrailingStopOrderRequest,
)
from alpaca.trading.enums import (
    AssetExchange,
    AssetStatus,
    OrderClass,
    OrderSide,
    OrderType,
    QueryOrderStatus,
    TimeInForce,
)


load_dotenv()

nest_asyncio.apply() 

stream = TradingStream(
    api_key=os.getenv("APCA-API-KEY-ID"),
    secret_key=os.getenv("APCA-API-SECRET-KEY"),
    paper=True,
)


async def handle_trade_updates(data):
    # print(type(data))
    # print(data)
    print("before mongo update")
    mongo.update_order_from_websocket(order_update=data,market_price=data.price)
    print("after mongo update")
    # Process the trade update here; find the trade form mongo, and update it
    # possibly do other things, depending on the trade status

stream.subscribe_trade_updates(handle_trade_updates)


async def start_stream():
    print("Stream listener started")
    await stream.run()  # Use await instead of stream.run() directly

asyncio.create_task(start_stream())  # This will start the stream in the backgroun

print("Stream listener started")

API_KEY_ID = os.getenv("APCA-API-KEY-ID")
API_SECRET_KEY = os.getenv("APCA-API-SECRET-KEY")
API_SERVER_DOMAIN = os.getenv("APCA-URL")
paper = True

trade_client = TradingClient(
    api_key=API_KEY_ID,
    secret_key=API_SECRET_KEY,
    paper=paper,
)

# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
print(type(acct))

# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
print(acct_config)


# we will place orders which Alapca trading platform supports
# - order classes: simple, bracket, oco, oto
# - order types: market, limit, stop, stop_limit, trailing_stop
#
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder
#
# we will also use fractional trading capability of Alpaca trading platform in this example
# ref. https://docs.alpaca.markets/docs/fractional-trading

# we will place orders for symbol: AAPL in this example
symbol = "TSLA"

# simple, market order, fractional qty
# Alpaca trading platform support fractional trading by default
# you can specify:
# fractional qty (e.g. 0.01 qty) in the order request (which is shown in this example)
# or notional value (e.g. 100 USD) (which is in the next example)
#
# If you have an error of `qty must be integer`,
# please try to `Reset Account` of your paper account via the Alpaca Trading API dashboard
# preparing market order
market_order_data = MarketOrderRequest(
    symbol=symbol, qty=5, side=OrderSide.BUY, time_in_force=TimeInForce.FOK
)
res = trade_client.submit_order(order_data=market_order_data)
mongo.insert_order(order=res, customer_id="3bean")


market_order_data = MarketOrderRequest(
    symbol=symbol, qty=5, side=OrderSide.SELL, time_in_force=TimeInForce.FOK
)
res = trade_client.submit_order(order_data=market_order_data)
mongo.insert_order(order=res, customer_id="3bean")


# resType = type(res)
# print(type(res))
# print(res)
# print(
#     "The submit response is of type: "
#     + resType
#     + " and has the following attributes:\n"
# )

# print(dir(res))

# preparing limit order
# limit_order_data = LimitOrderRequest(
#     symbol="BTC/USD",
#     limit_price=17000,
#     notional=4000,
#     side=OrderSide.SELL,
#     time_in_force=TimeInForce.FOK,
# )

# # Limit order
# limit_order = trade_client.submit_order(order_data=limit_order_data)



In [ ]:
# Required Imports
import os
from dotenv import load_dotenv
# Required Imports
from datetime import datetime
from pymongo import MongoClient, errors
import uuid


load_dotenv()


try:
    client = MongoClient(os.getenv("MONGO_URL"), serverSelectionTimeoutMS=5000)
    db = client["trading_db"]
    orders_collection = db["orders"]
    # Check the connection
    client.admin.command("ping")
    print("Connected to MongoDB successfully.")
except errors.ConnectionFailure as e:
    print(f"MongoDB connection failed: {e}")
    exit(1)


# Utility Functions
def datetime_to_unix(dt):
    """Convert datetime object to Unix timestamp (seconds)."""
    return int(dt.timestamp()) if dt else None


def calculate_transaction_duration(created_at, filled_at):
    """Calculate transaction duration in seconds."""
    if created_at and filled_at:
        return filled_at - created_at
    return None


def calculate_net_profit_loss(filled_qty, filled_avg_price, market_price):
    """Calculate net profit/loss for a filled order. Return a positive value for profit and a negative value for loss."""
    try:
        filled_qty = float(filled_qty)  # Ensure it's a float (to support fractional shares)
        filled_avg_price = float(filled_avg_price)  # Ensure it's a float
        market_price = float(market_price)  # Ensure it's a float
        return round((market_price - filled_avg_price) * filled_qty, 2)
    except (ValueError, TypeError) as e:
        print(f"Error calculating net profit/loss: {e}")
        return None


# Insert Order Function with Error Handling
def insert_order(order, customer_id):
    """Insert a new order document into MongoDB."""
    try:
        order_document = {
            "order_id": str(order.id),
            "client_order_id": str(order.client_order_id),
            "customer_id": customer_id,
            "timestamps": {
                "created_at": datetime_to_unix(order.created_at),
                "updated_at": datetime_to_unix(order.updated_at),
                "submitted_at": datetime_to_unix(order.submitted_at),
                "filled_at": datetime_to_unix(order.filled_at),
                "expired_at": datetime_to_unix(order.expired_at),
                "canceled_at": datetime_to_unix(order.canceled_at),
                "failed_at": datetime_to_unix(order.failed_at),
                "expires_at": datetime_to_unix(order.expires_at)
            },
            "asset": {
                "asset_id": str(order.asset_id),
                "symbol": order.symbol,
                "asset_class": order.asset_class.value
            },
            "order_details": {
                "qty": int(order.qty),
                "filled_qty": int(order.filled_qty),
                "filled_avg_price": float(order.filled_avg_price) if order.filled_avg_price else None,
                "price": None,
                "order_class": order.order_class.value,
                "order_type": order.order_type.value,
                "side": order.side.value,
                "time_in_force": order.time_in_force.value,
                "status": order.status.value,
                "position_qty": None,
                "transaction_duration": None,
                "net_profit_loss": None
            }
        }
        result = orders_collection.insert_one(order_document)
        print(f"Order inserted with ID: {result.inserted_id}")
    except errors.PyMongoError as e:
        print(f"Failed to insert order: {e}")


# Update Order from Websocket Function with Error Handling
def update_order_from_websocket(order_update, market_price=None):
    """Update an existing order document based on websocket data."""
    try:
        print(f"DEBUG: filled_qty={order_update['filled_qty']} (type={type(order_update['filled_qty'])})")
        print(f"DEBUG: filled_avg_price={order_update['filled_avg_price']} (type={type(order_update['filled_avg_price'])})")
        print(f"DEBUG: market_price={market_price} (type={type(market_price)})")
        
        query = {"order_id": str(order_update["id"])}
        
        filled_at_unix = datetime_to_unix(order_update["filled_at"])
        created_at_unix = datetime_to_unix(order_update.get("created_at"))
        transaction_duration = calculate_transaction_duration(created_at_unix, filled_at_unix)
        
        # Ensure values are converted to proper numeric types
        filled_qty = float(order_update["filled_qty"])
        filled_avg_price = float(order_update["filled_avg_price"])
        net_profit_loss = None
        
        if order_update["status"].value == "filled" and market_price:
            net_profit_loss = calculate_net_profit_loss(filled_qty, filled_avg_price, float(market_price))
            print(f"DEBUG: Calculated net_profit_loss={net_profit_loss}")
        
        update = {
            "$set": {
                "order_details.status": order_update["status"].value,
                "order_details.filled_qty": filled_qty,
                "order_details.filled_avg_price": filled_avg_price,
                "order_details.price": float(order_update["price"]),
                "order_details.position_qty": float(order_update["position_qty"]),
                "timestamps.updated_at": datetime_to_unix(order_update["updated_at"]),
                "timestamps.filled_at": filled_at_unix,
                "order_details.transaction_duration": transaction_duration,
                "order_details.net_profit_loss": net_profit_loss
            }
        }
        
        result = orders_collection.update_one(query, update)
        if result.modified_count > 0:
            print(f"Order {order_update['id']} updated successfully.")
        else:
            print(f"No order found with ID {order_update['id']}.")
    except Exception as e:
        print(f"Error in update_order_from_websocket: {e}")



# Example Usage (Mock Order and Websocket Update)
class MockOrder:
    id = uuid.uuid4()
    client_order_id = uuid.uuid4()
    created_at = datetime.utcnow()
    updated_at = datetime.utcnow()
    submitted_at = datetime.utcnow()
    filled_at = None
    expired_at = None
    canceled_at = None
    failed_at = None
    expires_at = datetime.utcnow()
    asset_id = uuid.uuid4()
    symbol = "AAPL"
    asset_class = type("AssetClass", (), {"value": "us_equity"})
    qty = 10
    filled_qty = 0.0
    filled_avg_price = None
    order_class = type("OrderClass", (), {"value": "simple"})
    order_type = type("OrderType", (), {"value": "market"})
    side = type("OrderSide", (), {"value": "buy"})
    time_in_force = type("TimeInForce", (), {"value": "day"})
    status = type("OrderStatus", (), {"value": "pending_new"})


order = MockOrder()
insert_order(order, 1)

# Mock Websocket Update Data (with made-up "filled" data)
mock_websocket_update = {
    "id": order.id,  # Keep as a UUID or string representing the order ID
    "status": type("OrderStatus", (), {"value": "filled"}),  # Status: filled
    "filled_qty": 10.5,  # Float to represent fractional shares
    "filled_avg_price": 220.0,  # Float for average price per share
    "price": 205.0,  # Float for current market price
    "position_qty": 130.0,  # Float for updated position quantity
    "updated_at": datetime.utcnow(),
    "filled_at": datetime.utcnow()
}

# Simulate Websocket Update Call
update_order_from_websocket(mock_websocket_update, market_price=float(mock_websocket_update.get("price", 0.0))
)

